# Charging Network Enhancement Optimization Model

This notebook implements and solves the mixed-integer linear programming model for optimizing the Kitchener-Waterloo charging network enhancement through:

1. Model Implementation
   - Decision variable definition
   - Constraint formulation
   - Objective function development
   - Gurobi model construction

2. Scenario Analysis
   - Base case optimization
   - Budget sensitivity analysis
   - Coverage requirement variations
   - Phasing strategy options

3. Solution Analysis
   - Result visualization
   - Cost-benefit analysis
   - Coverage improvement assessment
   - Implementation recommendations

4. Implementation Planning
   - Phased rollout strategy
   - Priority assignment
   - Resource allocation
   - Timeline development

This notebook provides the final optimization model and actionable recommendations for network enhancement.

## 1. Data Loading and Preprocessing

```python
# Station data
station_data = {
    'locations': [...],      # Existing station locations
    'types': [...],         # Current charger types
    'capacities': [...],    # Current number of ports
    'upgradeable': [...]    # Binary upgrade possibility
}

# Demand data
demand_data = {
    'population': [...],    # Population by area
    'ev_ownership': [...],  # EV ownership rates
    'growth_rate': [...]    # Projected EV adoption growth
}

# Cost parameters
cost_params = {
    'upgrade_costs': [...],  # L2 to L3 conversion costs
    'port_costs': [...],     # Additional port costs
    'operating_costs': [...] # Annual operating costs by type
}
```

## 2. Decision Variables

```python
# Primary Variables
x[i,j] = Binary variable for upgrading station i to type j
y[i] = Binary variable for adding new station at location i
z[i] = Binary variable for removing station i
p[i] = Integer variable for additional ports at location i

# Coverage Variables
c[i,j] = Binary variable for area i covered by station j
u[i] = Binary variable for area i being underserved
```

## 3. Constraints

### 3.1 Budget Constraints
```python
# Total cost constraint
sum(upgrade_costs[i] * x[i,j]) + sum(port_costs * p[i]) <= budget

# Maximum upgrades per period
sum(x[i,j]) <= max_upgrades_per_period
```

### 3.2 Coverage Constraints
```python
# Minimum population coverage
sum(population[i] * c[i,j]) >= min_coverage * total_population

# Maximum distance between L3 chargers
distance[i,j] * (x[i,'L3'] + x[j,'L3']) <= max_l3_distance
```

### 3.3 Infrastructure Constraints
```python
# Grid capacity
sum(power_demand[j] * x[i,j]) <= grid_capacity[i]

# Minimum ports per area
sum(p[i] for i in area[j]) >= min_ports[j]
```

### 3.4 Logical Constraints
```python
# Can't upgrade and remove same station
x[i,j] + z[i] <= 1

# Must be upgradeable for L2 to L3
x[i,'L3'] <= upgradeable[i]
```

## 4. Objective Function

```python
Maximize:
    # Coverage benefit
    w1 * sum(population[i] * ev_ownership[i] * c[i,j])
    
    # Infrastructure efficiency
    - w2 * sum(operating_costs[j] * x[i,j])
    
    # Upgrade benefit
    + w3 * sum(benefit_score[i] * x[i,'L3'])
    
    # Underservice penalty
    - w4 * sum(penalty[i] * u[i])
```

## 5. Model Implementation

```python
def build_optimization_model(data, params):
    """Build the Gurobi optimization model."""
    model = gp.Model("charging_network_enhancement")
    
    # Add variables
    x = model.addVars(...)  # Upgrade decisions
    y = model.addVars(...)  # New station decisions
    z = model.addVars(...)  # Removal decisions
    p = model.addVars(...)  # Port additions
    
    # Add constraints
    model.addConstrs(...)  # Budget constraints
    model.addConstrs(...)  # Coverage constraints
    model.addConstrs(...)  # Infrastructure constraints
    
    # Set objective
    model.setObjective(...)
    
    return model
```

## 6. Solution Analysis

```python
def analyze_solution(model, data):
    """Analyze optimization results."""
    results = {
        'upgrades': get_upgrade_decisions(model),
        'coverage': calculate_coverage_improvement(model),
        'costs': calculate_total_costs(model),
        'phasing': generate_implementation_phases(model)
    }
    return results
```

## 7. Sensitivity Analysis

```python
def run_sensitivity_analysis(model, params):
    """Run sensitivity analysis on key parameters."""
    scenarios = {
        'budget': test_budget_sensitivity(),
        'coverage': test_coverage_requirements(),
        'costs': test_cost_variations()
    }
    return scenarios
```

## 8. Implementation Planning

```python
def create_implementation_plan(results):
    """Generate phased implementation plan."""
    plan = {
        'phase1': identify_immediate_upgrades(),
        'phase2': plan_medium_term_changes(),
        'phase3': outline_long_term_strategy()
    }
    return plan
```

In [1]:
# Setup and imports
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import json
import matplotlib.pyplot as plt
import seaborn as sns
from src.data.utils import *

# Load optimization inputs
print("Loading optimization inputs...")
opt_data = load_latest_file(DATA_PATHS['optimization_inputs'], 'json')

Loading optimization inputs...


## 1. Model Parameters and Sets

In [2]:
def prepare_model_parameters():
    """Prepare parameters for optimization model."""
    stations = pd.DataFrame(opt_data['stations'])
    
    # Station sets
    station_sets = {
        'all_stations': stations.index.tolist(),
        'upgradeable': stations[stations['can_upgrade']].index.tolist(),
        'l2_stations': stations[stations['current_type'] == 'Level 2'].index.tolist(),
        'l3_stations': stations[stations['current_type'] == 'Level 3'].index.tolist()
    }
    
    # Cost parameters
    costs = {
        'upgrade': 50000,     # Cost to upgrade L2 to L3
        'new_port_l2': 5000,  # Cost per new L2 port
        'new_port_l3': 15000, # Cost per new L3 port
        'operating_l2': 2000, # Annual operating cost L2
        'operating_l3': 5000  # Annual operating cost L3
    }
    
    # Coverage parameters
    coverage = {
        'target_l3': opt_data['coverage']['target_l3'],
        'max_distance': opt_data['coverage']['max_distance'],
        'current_l3': opt_data['coverage']['current_l3']
    }
    
    return station_sets, costs, coverage

## 2. Decision Variables

In [3]:
def create_model_variables(model, station_sets):
    """Create optimization model variables."""
    vars = {}
    
    # Upgrade decisions (binary)
    vars['upgrade'] = model.addVars(
        station_sets['upgradeable'],
        vtype=GRB.BINARY,
        name="upgrade"
    )
    
    # Additional ports (integer)
    vars['new_ports'] = model.addVars(
        station_sets['all_stations'],
        vtype=GRB.INTEGER,
        name="new_ports"
    )
    
    return vars

## 3. Constraints

In [4]:
def add_model_constraints(model, vars, station_sets, params):
    """Add constraints to optimization model."""
    
    # Budget constraint
    model.addConstr(
        (gp.quicksum(params['costs']['upgrade'] * vars['upgrade'][i] 
                     for i in station_sets['upgradeable']) +
         gp.quicksum(params['costs']['new_port_l2'] * vars['new_ports'][i] 
                     for i in station_sets['l2_stations']) +
         gp.quicksum(params['costs']['new_port_l3'] * vars['new_ports'][i]
                     for i in station_sets['l3_stations']))
        <= params['budget'],
        name="budget"
    )
    
    # Coverage constraints, capacity constraints, etc.
    # [Additional constraints...]

## 4. Objective Function

In [5]:
def set_objective_function(model, vars, params):
    """Set the multi-objective optimization function."""
    
    # Coverage benefit
    coverage_benefit = gp.quicksum(...)
    
    # Operating costs
    operating_costs = gp.quicksum(...)
    
    # Set objective
    model.setObjective(
        coverage_benefit - operating_costs,
        sense=GRB.MAXIMIZE
    )

## 5. Model Solution

In [6]:
def solve_optimization_model():
    """Create and solve the optimization model."""
    
    # Create model
    model = gp.Model("charging_network_enhancement")
    
    # Prepare parameters
    station_sets, costs, coverage = prepare_model_parameters()
    
    # Create variables
    vars = create_model_variables(model, station_sets)
    
    # Add constraints
    add_model_constraints(model, vars, station_sets, params)
    
    # Set objective
    set_objective_function(model, vars, params)
    
    # Solve
    model.optimize()
    
    return model, vars

## 6. Solution Analysis

In [7]:
def analyze_solution(model, vars, station_sets):
    """Analyze optimization results."""
    if model.Status == GRB.OPTIMAL:
        results = {
            'upgrades': [i for i in station_sets['upgradeable'] 
                        if vars['upgrade'][i].X > 0.5],
            'new_ports': {i: vars['new_ports'][i].X 
                         for i in station_sets['all_stations']
                         if vars['new_ports'][i].X > 0}
        }
        
        return results
    else:
        print("No optimal solution found.")
        return None

## 7. Implementation Planning

In [8]:
def create_implementation_plan(results):
    """Create phased implementation plan from optimization results."""
    # [Implementation planning code...]